In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error

In [2]:
#import modelos
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

In [3]:
#filtrando los datos
df_train_original = pd.read_csv('train.csv', dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

df_test_original = pd.read_csv('test.csv', dtype={
    'tipodepropiedad':'category', 'ciudad':'category',\
    'provincia':'category'}, parse_dates=[16])

In [4]:
df_test_original.isnull().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
direccion                     13191
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                        9323
banos                          6554
metroscubiertos                4299
metrostotales                 12655
idzona                         7179
lat                           30695
lng                           30695
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [40]:
#filtracion general
df_test_original['fecha'] = pd.to_datetime(df_test_original['fecha'])
Id = df_test_original['id']
df_test = df_test_original.drop(columns=['fecha', 'id', 'provincia','titulo', 'idzona','descripcion', 'direccion', 'lat', 'lng']).copy()

df_test['mes'] = df_test_original['fecha'].dt.month
df_test['anio'] = df_test_original['fecha'].dt.year

df_test['cant_extras'] = sum([df_test['gimnasio'],df_test['usosmultiples'],df_test['piscina'],
                        df_test['escuelascercanas'],df_test['centroscomercialescercanos']])
df_test = df_test.drop(columns=['gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos'])
df_test.isnull().sum()


tipodepropiedad        7
ciudad                83
antiguedad         10714
habitaciones        5628
garages             9323
banos               6554
metroscubiertos     4299
metrostotales      12655
mes                    0
anio                   0
cant_extras            0
dtype: int64

In [6]:
#filtrando los precios
df_train_ciudad = df_train_original.groupby('ciudad').agg({'precio':'mean'}).sort_values('precio').reset_index()
df_train_ciudad['precio'] = np.log(df_train_ciudad['precio'])
df_train_ciudad
ciudad_log_precio = df_train_ciudad.set_index('ciudad').to_dict()
ciudad_log_precio = ciudad_log_precio['precio']
ciudad_log_precio

{'Cuaxomulco': 12.644327576461329,
 'Ecatzingo': 12.644327576461329,
 'Ojinaga': 12.6485123564297,
 'Santa Isabel Xiloxoxtla': 12.654597243098785,
 'Tecamachalco': 12.66032791780777,
 'Abalá': 12.66032791780777,
 'Rafael Lara Grajales': 12.676076274775909,
 'Donato Guerra': 12.676076274775909,
 'Ahuazotepec': 12.676076274775909,
 'Atlangatepec': 12.676076274775909,
 'Tolcayuca': 12.697715449879391,
 'Higueras': 12.711802306907794,
 'Cansahcab': 12.718896271958837,
 'Paracho': 12.736700896592344,
 'Tecali de Herrera': 12.765688433465597,
 'Oluta': 12.765688433465597,
 'Coquimatlán': 12.765688433465597,
 'Mazatepec': 12.765688433465597,
 'Sabinas': 12.77719342507992,
 'Gustavo Díaz Ordaz': 12.793859310432293,
 'Tixtla de Guerrero': 12.832011076396668,
 'Villa Purificación': 12.837344422372032,
 'Mariano Escobedo': 12.839667519762873,
 'Aconchi': 12.87261914429294,
 'Teolocholco': 12.875366147210718,
 'Acayucan': 12.885992732509024,
 'Escárcega': 12.886641043883259,
 'Calcahualco': 12.891

In [7]:
## funcion para setear los superficies
def set_metros(row):
    total = row.metrostotales
    covered = row.metroscubiertos
    if np.isnan(total):
        row.metrostotales = covered
        return row
    if np.isnan(covered):
        row.metroscubiertos = total
        return row
    return row

In [8]:
df_test[['metrostotales', 'metroscubiertos']] = df_test[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)

In [9]:
df_test.isnull().sum()

tipodepropiedad        7
ciudad                83
antiguedad         10714
habitaciones        5628
garages             9323
banos               6554
metroscubiertos        0
metrostotales          0
mes                    0
anio                   0
cant_extras            0
dtype: int64

In [10]:
#filtrar los tipos y ciudades
num_cols_con_nulls = ['antiguedad', 'habitaciones', 'banos', 'garages']
cat_cols_con_nulls = ['tipodepropiedad', 'ciudad']

In [11]:
# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

# imputing on numerical data
df_test[num_cols_con_nulls] = num_imputer.fit_transform(df_test[num_cols_con_nulls])

In [12]:
from sklearn.base import TransformerMixin

class SeriesImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        If the Series is of dtype Object, then impute with the most frequent object.
        If the Series is not of dtype Object, then impute with the mean.  

        """
    def fit(self, X, y=None):
        self.fill = X.value_counts().index[0]
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [13]:
# define categorical imputer
cat_imputer = SeriesImputer()

# imputing on categorical data
df_test['ciudad'] = cat_imputer.fit_transform(df_test['ciudad'])
df_test['tipodepropiedad'] = cat_imputer.fit_transform(df_test['tipodepropiedad'])

In [14]:
df_test.isnull().sum()

tipodepropiedad    0
ciudad             0
antiguedad         0
habitaciones       0
garages            0
banos              0
metroscubiertos    0
metrostotales      0
mes                0
anio               0
cant_extras        0
dtype: int64

In [15]:
#One Hot Encoding
df_test_coding = pd.get_dummies(df_test, prefix = ['Tipo'], columns = ['tipodepropiedad'])

In [43]:
df_test_coding['ciudad_ordinal'] = df_test_coding.ciudad.map(ciudad_log_precio)
df_test_fin = df_test_coding.drop(columns = 'ciudad')
df_test_fin['ciudad_ordinal'] = cat_imputer.fit_transform(df_test_fin['ciudad_ordinal'])
valor_test = df_test_fin.copy().values
df_test_fin.isnull().sum()
df_test_fin.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,mes,anio,cant_extras,Tipo_Apartamento,...,Tipo_Otros,Tipo_Quinta Vacacional,Tipo_Rancho,Tipo_Terreno,Tipo_Terreno comercial,Tipo_Terreno industrial,Tipo_Villa,Tipo_Huerta,Tipo_Lote,ciudad_ordinal
0,29.000000,3.0,1.547605,4.0,300.0,300.0,7,2013,0.0,0,...,0,0,0,0,0,0,0,0,0,15.318816
1,8.152599,1.0,1.000000,1.0,67.0,67.0,10,2015,0.0,1,...,0,0,0,0,0,0,0,0,0,14.522957
2,0.000000,2.0,1.000000,2.0,87.0,100.0,5,2015,1.0,1,...,0,0,0,0,0,0,0,0,0,15.041345
3,2.000000,2.0,2.000000,2.0,86.0,86.0,4,2015,0.0,1,...,0,0,0,0,0,0,0,0,0,14.765551
4,10.000000,2.0,1.000000,1.0,80.0,76.0,8,2013,2.0,0,...,0,0,0,0,0,0,0,0,0,13.726393


In [17]:
#filtrar los datos de train
df_train_original['fecha'] = pd.to_datetime(df_train_original['fecha'])

#elimino tipo hospedaje y garage
df_train = df_train_original[(df_train_original['tipodepropiedad'] != 'Hospedaje')&(df_train_original['tipodepropiedad'] != 'Garage')]
#obtengo precios
precio = df_train['precio'].values
df_train = df_train.drop(columns=['fecha', 'id', 'provincia','titulo', 'idzona','descripcion', 'direccion', 'lat', 'lng','precio']).copy()

df_train['mes'] = df_train_original['fecha'].dt.month
df_train['anio'] = df_train_original['fecha'].dt.year

df_train['cant_extras'] = sum([df_train['gimnasio'],df_train['usosmultiples'],df_train['piscina'],
                        df_train['escuelascercanas'],df_train['centroscomercialescercanos']])
df_train = df_train.drop(columns=['gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos'])
df_train[['metrostotales', 'metroscubiertos']] = df_train[['metrostotales', 'metroscubiertos']].apply(set_metros, axis = 1)

df_train.isnull().sum()

tipodepropiedad       46
ciudad               372
antiguedad         43553
habitaciones       22470
garages            37765
banos              26219
metroscubiertos        0
metrostotales          0
mes                    0
anio                   0
cant_extras            0
dtype: int64

In [18]:
#filtrar los tipos y ciudades
num_cols_con_nulls = ['antiguedad', 'habitaciones', 'banos', 'garages']

# define numerical imputer
num_imputer = SimpleImputer(strategy='mean')

# imputing on numerical data
df_train[num_cols_con_nulls] = num_imputer.fit_transform(df_train[num_cols_con_nulls])
# define categorical imputer
cat_imputer = SeriesImputer()

# imputing on categorical data
df_train['ciudad'] = cat_imputer.fit_transform(df_train['ciudad'])
df_train['tipodepropiedad'] = cat_imputer.fit_transform(df_train['tipodepropiedad'])
df_train['tipodepropiedad'].unique()

[Apartamento, Casa en condominio, Casa, Terreno, Terreno comercial, ..., Nave industrial, Rancho, Terreno industrial, Huerta, Lote]
Length: 22
Categories (22, object): [Apartamento, Casa en condominio, Casa, Terreno, ..., Rancho, Terreno industrial, Huerta, Lote]

In [19]:
#One Hot Encoding
df_train_coding = pd.get_dummies(df_train, prefix = ['Tipo'], columns = ['tipodepropiedad'])
df_train_coding['ciudad_ordinal'] = df_train_coding.ciudad.map(ciudad_log_precio)
df_train_fin = df_train_coding.drop(columns = ['ciudad','Tipo_Hospedaje','Tipo_Garage'] )
valor = df_train_fin.copy().values
df_train_fin.isnull().sum()

antiguedad                            0
habitaciones                          0
garages                               0
banos                                 0
metroscubiertos                       0
metrostotales                         0
mes                                   0
anio                                  0
cant_extras                           0
Tipo_Apartamento                      0
Tipo_Bodega comercial                 0
Tipo_Casa                             0
Tipo_Casa en condominio               0
Tipo_Casa uso de suelo                0
Tipo_Departamento Compartido          0
Tipo_Duplex                           0
Tipo_Edificio                         0
Tipo_Huerta                           0
Tipo_Inmuebles productivos urbanos    0
Tipo_Local Comercial                  0
Tipo_Local en centro comercial        0
Tipo_Lote                             0
Tipo_Nave industrial                  0
Tipo_Oficina comercial                0
Tipo_Otros                            0


In [33]:
precio_log = np.log(precio)
valor_train, valor_test, precio_train_log, precio_test_log = train_test_split(valor, precio_log, test_size=0.25, random_state=17)

In [35]:
#randomforest
random_forest = RandomForestRegressor(n_estimators=200, max_depth=1000, n_jobs=-1)
random_forest.fit(valor_train, precio_train_log)
precio_pred=random_forest.predict(valor_test)
mean_absolute_error(np.exp(precio_pred), np.exp(precio_test_log))

599716.87491837051

In [38]:
xgboost = XGBRegressor(max_depth=1000,subsample=0.3,min_child_weight =5,gamma = 2)
xgboost.fit(valor_train, precio_train_log)
precio_pred=xgboost.predict(valor_test)
mean_absolute_error(np.exp(precio_pred), np.exp(precio_test_log))

[14:26:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


616398.01076249999

In [46]:
valor_pred=random_forest.predict(valor_test)
df_resultado = pd.DataFrame()
df_resultado['Id'] = Id
df_resultado['target'] = np.exp(valor_pred)
df_resultado.to_csv('resultado_version2.csv',index=False)